In [10]:
def extract_text_from_pic(t, page_num):
    # sort words in the pic based on their position
    # from top-right
    words = []
    for i in range(0,len(t)):
        tmp = []
        y = (t[i][0][2][1] + t[i][0][1][1]) / 2

        tmp.append([y, t[i][0][2][0]])
        tmp.append(t[i][1])
        words.append(tmp)

    words = sorted(words)

    # finding the words that are in the same line
    lines = []
    line = []
    pre = words[0][0][0]
    for word in words:
        diff = abs(word[0][0] - pre)

        if diff <= 10:
            line.append(word)
        else:
            lines.append(line)
            line =[word]
            pre = word[0][0]
    if line:
        lines.append(line)

    # sorting words in a line based on their column
    for line in lines:
        for word in line:
            word[0].reverse()
    for i in range(0, len(lines)):
        lines[i].sort(reverse=True)

    return lines

In [11]:
from tqdm import tqdm
from pdf2image import convert_from_path

from PIL import Image

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

from collections.abc import Sequence

import os
import tempfile

import easyocr
reader = easyocr.Reader(['fa','en'])

def pdf_to_word(pdf_path, **kwargs):
    """ 
    A way to use easyOCR for extracting
    texts from pdf file.
    
    Args:
        pdf_path (str): PDF file path.
        output_dir (str): Output directory.
    """
    
    pdf_name = pdf_path.split('/')[-1].split('.')[0]
    pages = convert_from_path(pdf_path, **kwargs)

    texts = []
    print(f'PDF is preparing to convert into document [#{len(pages)} pages]')
    for i, page in tqdm(enumerate(pages), position=0):

        with tempfile.TemporaryDirectory() as img_dir:
            img_name = f'{pdf_name}-{i+1}.jpg'
            img_path = os.path.join(img_dir, img_name)

            page.save(img_path, 'JPEG')

            text = reader.readtext(img_path)
            texts += extract_text_from_pic(text, i)
    
    return texts

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [12]:
def save_as_word(lines, output_dir, pdf_name):
    document = Document()
    style_normal = document.styles['Normal']
    font = style_normal.font
    font.name = 'Arial'
    font.rtl = True

    style_h1 = document.styles['Heading 1']
    font = style_h1.font
    font.name = 'Arial'
    font.rtl = True

    i = 0
    for line in lines:
        heading = document.add_heading(f'صفحه: {i+1}', level=1)
        heading.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        heading.style = document.styles['Heading 1']
        for text in line:       
            paragraph = document.add_paragraph(str(text[1]))
            paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
            paragraph.style = document.styles['Normal']

        i += 1

    output_path = os.path.join(output_dir, f'{pdf_name}.docx')
    # output_path = os.path.join('./samples/', 'alaki.docx')
    document.save(output_path)
    print(f'Done! Your document can be find here "{output_path}"')

In [13]:
input_dir = input("Enter address of input file: ")
output_dir = input("Enter address of output file: ")

# input_dir = './samples/ahoy.pdf'
# output_dir = './samples/'

t = pdf_to_word(input_dir)

0it [00:00, ?it/s]

PDF is preparing to convert into document [#45 pages]


6it [01:16, 13.25s/it]

In [14]:
pdf_name = input_dir.split('/')[-1].split('.')[0]
save_as_word(t, output_dir, pdf_name)

Done! Your document can be find here "./samples/ahoy.docx"


In [15]:
# find maximum width of a line
mx_line = 0
for i in range(0, len(t)):
    tmp = t[i][0][2][1] - t[i][0][1][1]; 
    if tmp > mx_line:
        mx_line = tmp
mx_line

IndexError: list index out of range

In [16]:
# save all the lines in one string
lines = t
kol_text = ''
for line in lines:
    for text in line:
        kol_text += str(text[1]) + ' '

kol_text

'ب نام خدا ٧LTRAs ٥UNDs 2 ٥٢ ٦ 21 5P, را هنمای استفادد NOVINMED کت مهندسی پزشکی نویسن ( سهامی خاص ) نمایشگاه و دفترفروش مرکزی : تهران - خیابان مطهری - خيابان میرعماد - کوچه 11 ( پیمانی ) طبقه $3 واحد 111 تلفن 41223 فکس 88736222 دفتر اصفهان خیابان چهارباغ بالا مجتمع پارسیان شمارۀ 510 تلفن 031-36267172 فکس 031-36267363 كارخانه اصفهان شهرک صنعتی برق الکتر زونیک خمینی شهر - بلوک تلفن رسیدً شکایات : 031-95019155 تلفن خدمات پس از فروش : 31311544 -031 WWW novinmed com 1286.08 N٥VIN اهنماى استفاده Rev Signature Date of Issue Approved by Confirmed by Prepared by Issue Details No. ٨٥ J٠» Agharezaei 22/10/2018 Khorasani Azimian H Initial Release M AU UoKr 16/05/2019 Khorasani Shaabani Kh Bagheri Adding EMC Tabel A٥ Jopn Amending Serial 05/27/2020 Khorasani A Shaabani Kh Bagheri Label Adding warning list, ٨٥ J٠l»t Serial Label  EMC 05/19/2021 Khorasani Khorasani Bagheri Declaration  2 ULIRASOUNDS 215P, 21OP N٥VIN اهنماى استفاده فهرست مندرجات مقدمه توضیح علایم 5 موارد کاربرد عدم کاربرد اجزاء تنظیم 

In [ ]:
# save a string in word file
document = Document()
style_normal = document.styles['Normal']
font = style_normal.font
font.name = 'Arial'
font.rtl = True

style_h1 = document.styles['Heading 1']
font = style_h1.font
font.name = 'Arial'
font.rtl = True

paragraph = document.add_paragraph(kol_text)
paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
paragraph.style = document.styles['Normal']

output_path = os.path.join('./samples/', 'alaki.docx')
document.save(output_path)
print(f'Done! Your document can be find here "{output_path}"')